In [17]:
#Start by downloading GRUMP dataset and current SMOD dataset

In [1]:
import rasterio
from rasterio import features
from rasterio.features import shapes
from shapely.geometry import shape
import geopandas as gpd
from pandas import DataFrame

SMOD CLASSES
— Class 30: “Urban Centre grid cell”, if the cell belongs to an Urban Centre spatial entity;
— Class 23: “Dense Urban Cluster grid cell”, if the cell belongs to a Dense Urban Cluster spatial entity;
— Class 22: “Semi-dense Urban Cluster grid cell”, if the cell belongs to a Semi-dense Urban Cluster spatial entity;
— Class 21: “Suburban or per-urban grid cell”, if the cell belongs to an Urban Cluster cells at first hierarchical level but is not part of a Dense or Semi-dense Urban Cluster;
— Class 13: “Rural cluster grid cell”, if the cell belongs to a Rural Cluster spatial entity;
— Class 12: “Low Density Rural grid cell”, if the cell is classified as Rural grid cells at first hierarchical
level, has more than 50 inhabitant and is not part of a Rural Cluster;
— Class 11: “Very low density rural grid cell”, if the cell is classified as Rural grid cells at first hierarchical level, has less than 50 inhabitant and is not part of a Rural Cluster;
— Class 10: “Water grid cell”, if the cell has 0.5 share covered by permanent surface water and is not populated nor built.

In [4]:

#Open the SMOD raster
with rasterio.open(r'/Users/guilhermeiablonovski/Dropbox (SDSN)/SDG Geospatial Indicators Project/sdg-roadinfrastructure/_raw_data/GHS_SMOD_E2020_GLOBE_R2022A_54009_1000_V1_0/GHS_SMOD_E2020_GLOBE_R2022A_54009_1000_V1_0.tif') as src:
    smod_raw = src.read(1)

#Create a rasterio feature object by ingesting the land-cover raster
shapes = features.shapes(smod_raw,transform=src.transform)

#Create empty lists to store the values
urban = []
geometry = []

#Populate the lists with the geometries and values from the feature object
for shapedict, value in shapes:
    urban.append(value)
    geometry.append(shape(shapedict))
    
#Create a geodataframe with the values from the lists
gdf = gpd.GeoDataFrame(
    {'smod': urban, 'geometry': geometry },
    crs="ESRI:54009")

#Filter the features that correspond to the urban category
classes = [30,23,22,21]
urban = gdf[gdf["smod"].isin(classes)]
#urban = urban.to_crs("EPSG:4326")


#Plot the results
output_folder = '/Users/guilhermeiablonovski/Dropbox (SDSN)/SDG Geospatial Indicators Project/sdg-roadinfrastructure/_raw_data/GHS_SMOD_E2020_GLOBE_R2022A_54009_1000_V1_0/'
urban.to_file(output_folder+'SMOD_urban.gpkg')

In [6]:
#Open the GRUMP data
grump = gpd.read_file('/Users/guilhermeiablonovski/Dropbox (SDSN)/SDG Geospatial Indicators Project/sdg-roadinfrastructure/_raw_data/grump-v1-urban-ext-polygons-rev02-shp/grump-v1-urban-ext-polygons-rev02-shp/global_urban_extent_polygons_v1.01.shp')
smod = gpd.read_file('/Users/guilhermeiablonovski/Dropbox (SDSN)/SDG Geospatial Indicators Project/sdg-roadinfrastructure/_raw_data/GHS_SMOD_E2020_GLOBE_R2022A_54009_1000_V1_0/SMOD_urban.gpkg')

In [13]:
#Union datasets #Also can be done with QGIS to avoid projection mishandling
urban = grump.to_crs('ESRI:54009').overlay(smod, how='union')
urban.to_file('/Users/guilhermeiablonovski/Dropbox (SDSN)/SDG Geospatial Indicators Project/sdg-roadinfrastructure/_raw_data/SMOD_GRUMP_Urban.gpkg')

/Users/guilhermeiablonovski/opt/anaconda3/envs/geopandasenv/lib/python3.7/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [14]:
#JOIN FEATURES (Currently done with QGIS)
#Get the countries shape
world_countries = gpd.read_file('/Users/guilhermeiablonovski/Dropbox (SDSN)/world-administrative-boundaries-sdsn/world-administrative-boundaries-sdsn.shp')
urban = gpd.read_file('/Users/guilhermeiablonovski/Dropbox (SDSN)/SDG Geospatial Indicators Project/sdg-roadinfrastructure/_raw_data/SMOD_GRUMP_Urban_QGIS.gpkg')
rural = world_countries.overlay(urban, how='difference')